<a href="https://colab.research.google.com/github/kenNakaJJ/Ryu/blob/main/with.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
from gspread_dataframe import set_with_dataframe
from google.auth import default
import time
import pandas as pd
import numpy as np
import datetime
import gspread

# ==============================================================================
# ▼▼▼ 設定項目 ▼▼▼
# ==============================================================================
# --- スプレッドシートURL ---
# コーチの時給データが存在するスプレッドシート
SOURCE_WAGE_URL = "https://docs.google.com/spreadsheets/d/1pQUvhHjg3_dVnpqiZsHgoaJqg_UhWAs0yZ1LKHO6RVM/edit"
# メインの処理対象となるスプレッドシート
TARGET_SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1IiD3XqHt3NRcNGybEo6JPCmUGE0BbPQ7RQiZhkwGKR4/edit"

# --- シート名 ---
LOOKER_SHEET_NAME = "looker"
MASTER_SHEET_NAME = "マスター"
SUMMARY_SHEET_NAME = "集計"
FINAL_REPORT_SHEET_NAME = "集計結果（月別顧客別）"

# --- 数式 ---
# lookerシートのA1セルに設定するimportrange数式
IMPORTRANGE_FORMULA = '=importrange("https://docs.google.com/spreadsheets/d/1jVkup5Q4maoF1UW_-J1cAUXyFQpTvnHwYF6eOa-BgJI/edit","Looker!A1:h")'

# --- 列名 ---
# cシートとlookerシートで共通して使用する列名
COL_REPORTER = '報告者'
COL_PROCESS_CATEGORY = 'プロセス分類'
COL_WORK_TIME = '作業時間'
COL_BILLING_MONTH = '記帳該当月'
COL_CUSTOMER_NO = '顧客番号'
# 計算によって生成される列名
COL_STD_TIME = '標準時間'
COL_HOURLY_WAGE = '時給'
COL_SALARY = '給与'
COL_DIFF_MINUTES = '差（分）'
COL_BILLING = '請求'
COL_CUSTOMER_NAME = '顧客名'
COL_TASK_DETAIL = 'タスク詳細'

# ==============================================================================
# ヘルパー関数
# ==============================================================================

def get_gspread_client():
    """Google Colab環境で認証を行い、gspreadクライアントを返す"""
    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        print("✅ Google スプレッドシートへの認証が完了しました。")
        return gc
    except Exception as e:
        print(f"❌ 認証に失敗しました: {e}")
        return None

def get_unique_header(header_list):
    """ヘッダーリスト内の重複や空欄を処理して、一意なヘッダーリストを返す"""
    new_header = []
    seen = {}
    for i, col in enumerate(header_list):
        col_name = col.strip() if col.strip() else f"Unnamed_{i}"
        if col_name in seen:
            seen[col_name] += 1
            new_header.append(f"{col_name}.{seen[col_name]}")
        else:
            seen[col_name] = 0
            new_header.append(col_name)
    return new_header

def load_worksheet_as_df(worksheet):
    """ワークシートを読み込み、ヘッダーを処理してDataFrameとして返す"""
    all_values = worksheet.get_all_values()
    if not all_values:
        return pd.DataFrame()
    header = get_unique_header(all_values[0])
    data = all_values[1:]
    return pd.DataFrame(data, columns=header)

# ==============================================================================
# 各処理ステップの関数
# ==============================================================================

def step1_update_master_with_wages(gc):
    """[ステップ1] 最新のコーチ時給をマスターシートに転記する"""
    print("\n--- [ステップ1] 最新のコーチ時給をマスターシートに転記 ---")

    # URL設定の確認
    if SOURCE_WAGE_URL == "YOUR_SOURCE_SPREADSHEET_URL_HERE":
        print("❌ SOURCE_WAGE_URLが設定されていません。実際のスプレッドシートURLを設定してください。")
        return False
    if TARGET_SPREADSHEET_URL == "YOUR_TARGET_SPREADSHEET_URL_HERE":
        print("❌ TARGET_SPREADSHEET_URLが設定されていません。実際のスプレッドシートURLを設定してください。")
        return False

    try:
        print(f"ソーススプレッドシートを開いています: {SOURCE_WAGE_URL}")
        sh_source = gc.open_by_url(SOURCE_WAGE_URL)
        latest_date = None
        latest_sheet = None
        for sheet in sh_source.worksheets():
            try:
                sheet_date = datetime.datetime.strptime(f"20{sheet.title}", "%Y/%m").date()
                if latest_date is None or sheet_date > latest_date:
                    latest_date = sheet_date
                    latest_sheet = sheet
            except ValueError:
                continue

        if latest_sheet is None:
            raise ValueError("日付形式（YY/MM）のシートが見つかりませんでした。")
        print(f"✅ 最新の時給シート '{latest_sheet.title}' を特定しました。")

        df_source = load_worksheet_as_df(latest_sheet)
        if df_source.empty or len(df_source.columns) < 44:
             raise ValueError("ソースシートの列数が不足しているか、データが空です。")
        staff_col_index = 2 # C列
        wage_col_index = 43 # AR列
        df_to_write = df_source.iloc[:, [staff_col_index, wage_col_index]].copy()
        df_to_write.columns = ['スタッフ名', '時給'] # ヘッダーを明確にする
        df_to_write = df_to_write.iloc[4:].reset_index(drop=True) # 5行目からをデータとして扱う

        sh_dest = gc.open_by_url(TARGET_SPREADSHEET_URL)
        ws_master = sh_dest.worksheet(MASTER_SHEET_NAME)

        # マスターシートの既存データを読み込み
        master_all_values = ws_master.get_all_values()
        if not master_all_values:
             raise ValueError("マスターシートが空です。")

        # 既存のマスターデータから、時給データを更新する部分を特定
        # 例として、スタッフ名の列（D列）と時給の列（E列）を更新対象とする
        master_staff_col_index = 3 # D列 (0始まり)
        master_wage_col_index = 4 # E列 (0始まり)

        # 更新用のデータリストを作成
        update_data = []
        # 既存のマスターデータを行ごとに処理
        for row_index, row in enumerate(master_all_values):
            if row_index == 0: # ヘッダー行はスキップ、または必要に応じて処理
                # ヘッダー行が必要な場合はここで処理
                continue
            if len(row) > master_staff_col_index:
                staff_name = row[master_staff_col_index].strip()
                if staff_name:
                    # 最新の時給データからスタッフ名で検索
                    wage_row = df_to_write[df_to_write['スタッフ名'] == staff_name]
                    if not wage_row.empty:
                        # 該当するスタッフが見つかった場合、最新の時給を取得
                        latest_wage = wage_row.iloc[0]['時給']
                        # 更新対象の行データを作成（E列のみ更新）
                        # 行の長さを確認し、必要に応じて拡張
                        current_row_length = len(row)
                        if current_row_length <= master_wage_col_index:
                            row.extend([''] * (master_wage_col_index - current_row_length + 1))
                        row[master_wage_col_index] = latest_wage
                    # 更新データリストに追加
                    update_data.append(row)
                else:
                     # スタッフ名が空の場合は既存の行をそのまま追加（またはスキップ）
                     update_data.append(row)
            else:
                 # D列が存在しない行は既存の行をそのまま追加
                 update_data.append(row)

        # 更新データをマスターシートに書き込み（シート全体を更新）
        # 既存のシートをクリアして新しいデータで書き込む
        # ws_master.clear() # クリアしないことで既存の数式などを保持
        ws_master.update([get_unique_header(master_all_values[0])] + update_data, range_name='A1') # 元のヘッダーを保持してA1から更新

        print(f"✅ マスターシートのD列とE列に最新の時給データを書き込みました。")
        return True
    except Exception as e:
        import traceback
        print(f"❌ ステップ1でエラーが発生しました: {e}")
        print("詳細なエラー情報:")
        traceback.print_exc()
        return False

def step2_reset_looker_sheet(sh):
    """[ステップ2] lookerシートをクリアし、importrange数式を再設定する"""
    print(f"\n--- [ステップ2] '{LOOKER_SHEET_NAME}' シートをIMPORTRANGEで更新 ---")
    try:
        worksheet = sh.worksheet(LOOKER_SHEET_NAME)
        worksheet.clear()
        # worksheet.update('A1', [[IMPORTRANGE_FORMULA]], raw=False) # DeprecationWarning 対応
        worksheet.update(range_name='A1', values=[[IMPORTRANGE_FORMULA]], raw=False)
        print(f"✅ '{LOOKER_SHEET_NAME}' シートをクリアし、A1セルに数式を再設定しました。")
        return True
    except Exception as e:
        print(f"❌ ステップ2でエラーが発生しました: {e}")
        return False

def step3_prepare_c_sheets_columns(sh):
    """[ステップ3] cシート群に必要な列が存在するか確認し、なければ追加する"""
    print("\n--- [ステップ3] 'c'で始まるシートの列整備 ---")
    try:
        print("IMPORTRANGEの反映を待機しています...")
        time.sleep(10) # IMPORTRANGEの反映待機時間を延長
        looker_ws = sh.worksheet(LOOKER_SHEET_NAME)
        looker_header = get_unique_header(looker_ws.row_values(1))

        required_cols_set = set(looker_header + [COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY])

        c_sheets = [ws for ws in sh.worksheets() if ws.title.startswith('c')]
        for c_sheet in c_sheets:
            c_header = c_sheet.row_values(1)
            c_header_set = set(get_unique_header(c_header))
            missing_cols = list(required_cols_set - c_header_set)

            if missing_cols:
                print(f"シート '{c_sheet.title}' に不足列 {missing_cols} を追加します。")
                # 現在の列数を取得
                current_col_count = c_sheet.col_count
                # 不足列を追加
                c_sheet.add_cols(len(missing_cols))
                # ヘッダーを更新
                for i, col_name in enumerate(missing_cols):
                    # 新しい列のヘッダーを設定するセルの位置を計算
                    header_col_index = current_col_count + i + 1
                    c_sheet.update_cell(row=1, col=header_col_index, value=col_name)

        print("✅ 全ての'c'シートの列整備が完了しました。")
        return True
    except Exception as e:
        print(f"❌ ステップ3でエラーが発生しました: {e}")
        return False

def step4_calculate_and_update_c_sheets(sh):
    """[ステップ4] マスターデータを基に、cシート群の計算を行い更新する"""
    print("\n--- [ステップ4] 'c'シートの計算と更新 ---")
    try:
        ws_master = sh.worksheet(MASTER_SHEET_NAME)
        df_master = load_worksheet_as_df(ws_master)

        # 標準時間ルックアップを作成 (プロセス分類 -> 標準時間)
        # マスターシートのA列(プロセス分類)とB列(標準時間)を使用
        std_time_lookup = pd.Series(
            pd.to_numeric(df_master.iloc[:, 1], errors='coerce').values,
            index=df_master.iloc[:, 0]
        ).to_dict()
        print(f"✅ 標準時間ルックアップを作成しました ({len(std_time_lookup)}件)。")

        # 時給ルックアップを作成 (スタッフ名 -> 時給)
        # マスターシートのD列(スタッフ名)とE列(時給)を使用
        # E列の時給は文字列の場合があるので、数値に変換し、無効な値はNaNにする
        df_master_wage = df_master.iloc[:, [3, 4]].dropna().copy()
        df_master_wage.columns = ['スタッフ名', '時給'] # ヘッダーを明確にする
        df_master_wage['時給'] = df_master_wage['時給'].astype(str).str.replace('¥', '').str.replace(',', '').str.strip()
        wage_lookup = pd.Series(
            pd.to_numeric(df_master_wage['時給'], errors='coerce').values,
            index=df_master_wage['スタッフ名']
        ).to_dict()
        print(f"✅ 時給ルックアップを作成しました ({len(wage_lookup)}件)。")

        c_sheets = [ws for ws in sh.worksheets() if ws.title.startswith('c')]
        for c_sheet in c_sheets:
            print(f"  > シート '{c_sheet.title}' の処理中...")
            df_c = load_worksheet_as_df(c_sheet)
            if df_c.empty:
                print(f"    シート '{c_sheet.title}' は空です。スキップします。")
                continue

            # 必要な列が存在するか確認し、列インデックスを取得
            if COL_REPORTER not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_REPORTER}' 列がありません。スキップします。")
                 continue

            # 必要な計算列が存在するか確認し、なければエラーメッセージ
            if COL_STD_TIME not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_STD_TIME}' 列がありません。標準時間計算をスキップします。")
            if COL_WORK_TIME not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_WORK_TIME}' 列がありません。給与計算をスキップします。")
            if COL_HOURLY_WAGE not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_HOURLY_WAGE}' 列がありません。時給・給与計算をスキップします。")
                 continue # 時給列がないと給与も計算できないためスキップ

            # 数値変換 (エラーがあればNaN)
            df_c[COL_WORK_TIME] = pd.to_numeric(df_c[COL_WORK_TIME], errors='coerce')
            if COL_STD_TIME in df_c.columns:
                df_c[COL_STD_TIME] = pd.to_numeric(df_c[COL_STD_TIME], errors='coerce')

            # 時給をルックアップで取得し、DataFrameに代入
            df_c[COL_HOURLY_WAGE] = df_c[COL_REPORTER].map(wage_lookup)
            df_c[COL_HOURLY_WAGE] = pd.to_numeric(df_c[COL_HOURLY_WAGE], errors='coerce')
            print(f"    シート '{c_sheet.title}' の時給を取得しました。")

            # 給与を計算 (作業時間 * 時給 / 60, 小数点以下切り捨て)
            # NaNを含む行は計算しない
            valid_rows = df_c[COL_WORK_TIME].notna() & df_c[COL_HOURLY_WAGE].notna()
            df_c.loc[valid_rows, COL_SALARY] = np.floor(
                df_c.loc[valid_rows, COL_WORK_TIME] * df_c.loc[valid_rows, COL_HOURLY_WAGE] / 60
            )
            # 計算できなかった行 (NaN) は空白に設定
            df_c[COL_SALARY] = df_c[COL_SALARY].fillna('')
            print(f"    シート '{c_sheet.title}' の給与を計算しました。")

            # 差（分）の計算
            if all(c in df_c.columns for c in [COL_WORK_TIME, COL_STD_TIME]) and COL_DIFF_MINUTES in df_c.columns:
                df_c[COL_DIFF_MINUTES] = df_c[COL_WORK_TIME] - df_c[COL_STD_TIME]
                print(f"    シート '{c_sheet.title}' の差（分）を計算しました。")

            # 更新する列のみを選択して書き込み
            cols_to_update = [COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY]
            # 実際に存在する列のみを対象とする
            existing_cols_to_update = [col for col in cols_to_update if col in df_c.columns]

            if existing_cols_to_update:
                # 更新対象のDataFrameを作成（既存の列はそのまま、計算した列は更新）
                df_for_write = df_c.copy()
                # NaNを空白に変換して書き込みに備える
                df_for_write[existing_cols_to_update] = df_for_write[existing_cols_to_update].fillna('')

                # ヘッダー行を含めて、DataFrame全体を書き込み
                c_sheet.clear()
                set_with_dataframe(c_sheet, df_for_write, include_index=False)
                print(f"    シート '{c_sheet.title}' を更新しました。")
            else:
                 print(f"    シート '{c_sheet.title}' に更新対象の列がありませんでした。")

        print("✅ 全ての'c'シートの計算と更新が完了しました。")
        return True
    except Exception as e:
        print(f"❌ ステップ4でエラーが発生しました: {e}")
        return False

def step5_create_summary_sheet(sh):
    """[ステップ5] lookerとcシート群からデータを集約し、`集計`シートを作成する"""
    print(f"\n--- [ステップ5] '{SUMMARY_SHEET_NAME}' シートの作成 ---")
    try:
        ws_looker = sh.worksheet(LOOKER_SHEET_NAME)
        df_looker = load_worksheet_as_df(ws_looker)
        if not df_looker.empty and COL_BILLING_MONTH in df_looker.columns:
            # '記帳該当月' 列が数値に変換できる行のみを抽出
            df_looker_filtered = df_looker[pd.to_numeric(df_looker[COL_BILLING_MONTH], errors='coerce').notna()].copy()
        else:
            df_looker_filtered = pd.DataFrame(columns=df_looker.columns.tolist() + [COL_STD_TIME, COL_DIFF_MINUTES]) # 存在しない列を追加して空のDataFrameを作成
        print(f"✅ '{LOOKER_SHEET_NAME}' シートから {len(df_looker_filtered)} 件のデータを抽出しました。")

        all_c_dfs = []
        c_sheets = [ws for ws in sh.worksheets() if ws.title.startswith('c')]
        key_cols = [COL_REPORTER, COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_CUSTOMER_NAME, COL_PROCESS_CATEGORY, COL_TASK_DETAIL]
        sum_cols = [COL_WORK_TIME, COL_SALARY, COL_BILLING, COL_STD_TIME, COL_DIFF_MINUTES]

        for c_sheet in c_sheets:
            df_c = load_worksheet_as_df(c_sheet)
            if df_c.empty: continue
            if COL_BILLING_MONTH not in df_c.columns: continue
            # '記帳該当月' 列が数値に変換できる行のみを抽出
            df_c_filtered = df_c[pd.to_numeric(df_c[COL_BILLING_MONTH], errors='coerce').notna()].copy()
            if df_c_filtered.empty: continue

            # 集計対象列を数値に変換 (エラーはNaN)
            for col in sum_cols:
                if col in df_c_filtered.columns:
                    df_c_filtered[col] = pd.to_numeric(df_c_filtered[col], errors='coerce')

            # 存在するキー列のみを使用
            existing_key_cols = [col for col in key_cols if col in df_c_filtered.columns]
            if not existing_key_cols:
                print(f"⚠️ シート '{c_sheet.title}' に集計に必要なキー列がありません。スキップします。")
                continue

            # 集計辞書を作成（存在する集計対象列のみ）
            agg_dict = {col: 'sum' for col in sum_cols if col in df_c_filtered.columns}
            # 時給は平均値または最初の値など、適切な集計方法を選択（ここでは最初の値とする）
            if COL_HOURLY_WAGE in df_c_filtered.columns:
                 agg_dict[COL_HOURLY_WAGE] = 'first'

            # 存在するキー列と集計対象列のみを含むDataFrameを作成して集計
            cols_for_agg = existing_key_cols + list(agg_dict.keys())
            df_c_agg = df_c_filtered[cols_for_agg].groupby(existing_key_cols, as_index=False).agg(agg_dict)
            all_c_dfs.append(df_c_agg)

        # lookerデータとcシート群のデータを結合
        # 結合前に両方のDataFrameに必要な全列が存在するように調整
        all_cols = list(set(df_looker_filtered.columns.tolist() + [col for df in all_c_dfs for col in df.columns]))
        df_looker_aligned = df_looker_filtered.reindex(columns=all_cols)
        c_combined_aligned = pd.concat(all_c_dfs, ignore_index=True).reindex(columns=all_cols) if all_c_dfs else pd.DataFrame(columns=all_cols)

        df_summary = pd.concat([df_looker_aligned, c_combined_aligned], ignore_index=True)

        try:
            ws_summary = sh.worksheet(SUMMARY_SHEET_NAME)
            ws_summary.clear()
        except gspread.exceptions.WorksheetNotFound:
            ws_summary = sh.add_worksheet(title=SUMMARY_SHEET_NAME, rows="1000", cols="30")

        # NaNを空白に変換して書き込み
        df_for_write = df_summary.fillna('')
        set_with_dataframe(ws_summary, df_for_write, include_index=False)
        print(f"✅ '{SUMMARY_SHEET_NAME}' シートに {len(df_summary)} 件のデータを書き込みました。")
        return True
    except Exception as e:
        print(f"❌ ステップ5でエラーが発生しました: {e}")
        return False

def step6_create_final_report_sheet(sh):
    """[ステップ6] `集計`シートを基に`集計結果（月別顧客別）`シートを作成し、書式設定する"""
    print(f"\n--- [ステップ6] '{FINAL_REPORT_SHEET_NAME}' シートの作成と書式設定 ---")
    try:
        ws_summary = sh.worksheet(SUMMARY_SHEET_NAME)
        df_summary = load_worksheet_as_df(ws_summary)
        if df_summary.empty:
            print(f"⚠️ '{SUMMARY_SHEET_NAME}' シートが空のため、処理をスキップします。")
            return True

        # 集計に必要なキー列と集計対象列を定義
        group_cols = [COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_PROCESS_CATEGORY]
        sum_cols = [COL_WORK_TIME, COL_SALARY, COL_BILLING, COL_STD_TIME, COL_DIFF_MINUTES]
        # 時給も表示する列に含める
        display_cols_order = [COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_CUSTOMER_NAME, COL_PROCESS_CATEGORY, COL_TASK_DETAIL, COL_REPORTER, COL_WORK_TIME, COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY, COL_BILLING]

        # 実際に集計シートに存在するキー列と集計対象列を特定
        existing_group_cols = [c for c in group_cols if c in df_summary.columns]
        existing_sum_cols = [c for c in sum_cols if c in df_summary.columns]
        # 集計対象列以外の列も取得（小計行で表示するために必要）
        other_cols = [c for c in df_summary.columns if c not in existing_group_cols + existing_sum_cols]

        if not all(c in df_summary.columns for c in [COL_BILLING_MONTH, COL_CUSTOMER_NO]):
            raise ValueError("集計に必要なキー列（記帳該当月, 顧客番号）が集計シートにありません。")

        # 集計対象列を数値に変換 (エラーは0に変換)
        for col in existing_sum_cols:
             if col in df_summary.columns: # 念のため再確認
                df_summary[col] = pd.to_numeric(df_summary[col], errors='coerce').fillna(0)

        # 集計辞書を作成
        agg_dict = {col: 'sum' for col in existing_sum_cols}
        # 集計対象列以外の列は最初の値を使用
        for col in other_cols:
             if col in df_summary.columns: # 念のため再確認
                agg_dict[col] = 'first'

        # 存在するキー列でグループ化し、集計を実行
        df_aggregated = df_summary.groupby(existing_group_cols, as_index=False).agg(agg_dict)

        # 記帳該当月と顧客番号でソート
        sort_cols = [c for c in [COL_BILLING_MONTH, COL_CUSTOMER_NO] if c in df_aggregated.columns]
        if sort_cols:
            df_aggregated = df_aggregated.sort_values(by=sort_cols)

        # 小計行を作成して結合
        final_dfs = []
        # 存在するグループ列の組み合わせでループ
        existing_customer_group_cols = [c for c in [COL_BILLING_MONTH, COL_CUSTOMER_NO] if c in df_aggregated.columns]
        if existing_customer_group_cols:
            for name, group in df_aggregated.groupby(existing_customer_group_cols):
                final_dfs.append(group)
                # 小計行のデータを作成
                subtotal = group[existing_sum_cols].sum()
                # 小計行のキー列を設定
                subtotal_key_values = list(name) + ['小計'] * (len(existing_group_cols) - len(existing_customer_group_cols))
                subtotal_row_data = dict(zip(existing_group_cols, subtotal_key_values))
                # 集計対象列の値を設定
                subtotal_row_data.update(subtotal.to_dict())
                # その他の列はNaNで埋める（後でfillna('')で空白にする）
                subtotal_row_df = pd.DataFrame([subtotal_row_data]).reindex(columns=df_aggregated.columns)
                final_dfs.append(subtotal_row_df)
        else:
             # 顧客別グループ化ができない場合は、集計済みデータそのまま
             final_dfs.append(df_aggregated)

        df_final = pd.concat(final_dfs, ignore_index=True) if final_dfs else pd.DataFrame(columns=df_aggregated.columns)

        # 表示列の順序を調整
        existing_display_cols = [c for c in display_cols_order if c in df_final.columns]
        df_final = df_final[existing_display_cols]

        try:
            ws_final = sh.worksheet(FINAL_REPORT_SHEET_NAME)
            ws_final.clear()
        except gspread.exceptions.WorksheetNotFound:
            ws_final = sh.add_worksheet(title=FINAL_REPORT_SHEET_NAME, rows="1000", cols="30")

        # NaNを空白に変換して書き込み
        df_for_write = df_final.fillna('')
        set_with_dataframe(ws_final, df_for_write, include_index=False)
        print(f"✅ '{FINAL_REPORT_SHEET_NAME}' に {len(df_final)} 件のデータを書き込みました。")

        # --- 書式設定（APIエラー修正版） ---
        requests = []
        last_row = len(df_final) + 1 # ヘッダー行を含むため+1
        last_col = len(df_final.columns)

        # 1. 罫線設定のリクエスト
        border_style = {"style": "SOLID", "width": 1, "color": {"red": 0.0, "green": 0.0, "blue": 0.0}}
        # シート全体の罫線
        requests.append({
            "updateBorders": {
                "range": {"sheetId": ws_final.id, "startRowIndex": 0, "endRowIndex": last_row, "startColumnIndex": 0, "endColumnIndex": last_col},
                "top": border_style, "bottom": border_style, "left": border_style, "right": border_style,
                "innerHorizontal": border_style, "innerVertical": border_style,
            }
        })

        # 2. 小計行の背景色設定のリクエスト
        # 小計行のインデックスを特定 (ヘッダー行を考慮して+1)
        subtotal_rows_indices = [i + 1 for i, row in df_final.iterrows() if '小計' in str(row.get(COL_BILLING_MONTH, ''))]
        if subtotal_rows_indices:
            for row_index in subtotal_rows_indices:
                requests.append({
                    "repeatCell": {
                        "range": {"sheetId": ws_final.id, "startRowIndex": row_index, "endRowIndex": row_index + 1, "startColumnIndex": 0, "endColumnIndex": last_col}, # endRowIndexは排他的なので+1
                        "cell": {"userEnteredFormat": {"backgroundColor": {"red": 1.0, "green": 1.0, "blue": 0.8}}}, # 薄い黄色
                        "fields": "userEnteredFormat.backgroundColor"
                    }
                })

        # 3. リクエストをまとめて実行
        if requests:
            sh.batch_update({"requests": requests})
            print(f"✅ 書式設定（罫線と小計行の背景色）を適用しました。")

        return True
    except Exception as e:
        print(f"❌ ステップ6でエラーが発生しました: {e}")
        return False

# ==============================================================================
# メイン処理
# ==============================================================================
def main():
    """全ての処理を順番に実行するメイン関数"""
    print("====== データ処理を開始します ======")

    # URL設定の確認
    if SOURCE_WAGE_URL == "YOUR_SOURCE_SPREADSHEET_URL_HERE":
        print("❌ 設定エラー: SOURCE_WAGE_URLを実際のスプレッドシートURLに変更してください。")
        return
    if TARGET_SPREADSHEET_URL == "YOUR_TARGET_SPREADSHEET_URL_HERE":
        print("❌ 設定エラー: TARGET_SPREADSHEET_URLを実際のスプレッドシートURLに変更してください。")
        return
    if IMPORTRANGE_FORMULA == '=importrange("YOUR_LOOKER_SPREADSHEET_URL_HERE","Looker!A1:h")':
        print("❌ 設定エラー: IMPORTRANGE_FORMULAのURLを実際のスプレッドシートURLに変更してください。")
        return

    gc = get_gspread_client()
    if not gc: return

    if not step1_update_master_with_wages(gc): return

    try:
        sh = gc.open_by_url(TARGET_SPREADSHEET_URL)
    except Exception as e:
        print(f"❌ メインの処理対象スプレッドシートが開けません: {e}")
        return

    if not step2_reset_looker_sheet(sh): return
    if not step3_prepare_c_sheets_columns(sh): return
    if not step4_calculate_and_update_c_sheets(sh): return
    if not step5_create_summary_sheet(sh): return
    if not step6_create_final_report_sheet(sh): return

    print("\n🎉🎉🎉 全ての処理が正常に完了しました！ 🎉🎉🎉")
    print(f"最終確認はこちら: {TARGET_SPREADSHEET_URL}")

# --- スクリプトの実行 ---
main()

====== データ処理を開始します ======
✅ Google スプレッドシートへの認証が完了しました。

--- [ステップ1] 最新のコーチ時給をマスターシートに転記 ---
ソーススプレッドシートを開いています: https://docs.google.com/spreadsheets/d/1pQUvhHjg3_dVnpqiZsHgoaJqg_UhWAs0yZ1LKHO6RVM/edit
✅ 最新の時給シート '25/07' を特定しました。
✅ マスターシートのD列とE列に最新の時給データを書き込みました。

--- [ステップ2] 'looker' シートをIMPORTRANGEで更新 ---
✅ 'looker' シートをクリアし、A1セルに数式を再設定しました。

--- [ステップ3] 'c'で始まるシートの列整備 ---
IMPORTRANGEの反映を待機しています...
シート 'c大島' に不足列 ['差（分）', '顧客名', '標準時間', '作業時間', '請求', 'プロセス分類', '顧客番号', '記帳該当月', '時給', '給与', '報告者'] を追加します。
✅ 全ての'c'シートの列整備が完了しました。

--- [ステップ4] 'c'シートの計算と更新 ---
✅ 標準時間ルックアップを作成しました (29件)。
✅ 時給ルックアップを作成しました (32件)。
  > シート 'c大島' の処理中...
    シート 'c大島' の時給を取得しました。
    シート 'c大島' の給与を計算しました。
    シート 'c大島' の差（分）を計算しました。
    シート 'c大島' を更新しました。
  > シート 'c矢頭' の処理中...
    シート 'c矢頭' の時給を取得しました。
    シート 'c矢頭' の給与を計算しました。
    シート 'c矢頭' の差（分）を計算しました。
    シート 'c矢頭' を更新しました。
  > シート 'c尾崎' の処理中...
    シート 'c尾崎' の時給を取得しました。
    シート 'c尾崎' の給与を計算しました。
    シート 'c尾崎' の差（分）を計算しました。
    シート 'c尾崎' を更新しました。
  > シート 

In [ ]:
from google.colab import auth
from gspread_dataframe import set_with_dataframe
from google.auth import default
import time
import pandas as pd
import numpy as np
import datetime
import gspread

# ==============================================================================
# ▼▼▼ 設定項目 ▼▼▼
# ==============================================================================
# --- スプレッドシートURL ---
# コーチの時給データが存在するスプレッドシート
SOURCE_WAGE_URL = "https://docs.google.com/spreadsheets/d/1pQUvhHjg3_dVnpqiZsHgoaJqg_UhWAs0yZ1LKHO6RVM/edit"
# メインの処理対象となるスプレッドシート
TARGET_SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1IiD3XqHt3NRcNGybEo6JPCmUGE0BbPQ7RQiZhkwGKR4/edit"

# --- シート名 ---
LOOKER_SHEET_NAME = "looker"
MASTER_SHEET_NAME = "マスター"
SUMMARY_SHEET_NAME = "集計"
FINAL_REPORT_SHEET_NAME = "集計結果（月別顧客別）"

# --- 数式 ---
# lookerシートのA1セルに設定するimportrange数式
IMPORTRANGE_FORMULA = '=importrange("https://docs.google.com/spreadsheets/d/1jVkup5Q4maoF1UW_-J1cAUXyFQpTvnHwYF6eOa-BgJI/edit","Looker!A1:h")'

# --- 列名 ---
# cシートとlookerシートで共通して使用する列名
COL_REPORTER = '報告者'
COL_PROCESS_CATEGORY = 'プロセス分類'
COL_WORK_TIME = '作業時間'
COL_BILLING_MONTH = '記帳該当月'
COL_CUSTOMER_NO = '顧客番号'
# 計算によって生成される列名
COL_STD_TIME = '標準時間'
COL_HOURLY_WAGE = '時給'
COL_SALARY = '給与'
COL_DIFF_MINUTES = '差（分）'
COL_BILLING = '請求'
COL_CUSTOMER_NAME = '顧客名'
COL_TASK_DETAIL = 'タスク詳細'
# 新しく追加する列名
COL_PROFIT_LOSS = '損益'


# ==============================================================================
# ヘルパー関数
# ==============================================================================

def get_gspread_client():
    """Google Colab環境で認証を行い、gspreadクライアントを返す"""
    try:
        auth.authenticate_user()
        creds, _ = default()
        gc = gspread.authorize(creds)
        print("✅ Google スプレッドシートへの認証が完了しました。")
        return gc
    except Exception as e:
        print(f"❌ 認証に失敗しました: {e}")
        return None

def get_unique_header(header_list):
    """ヘッダーリスト内の重複や空欄を処理して、一意なヘッダーリストを返す"""
    new_header = []
    seen = {}
    for i, col in enumerate(header_list):
        col_name = col.strip() if col.strip() else f"Unnamed_{i}"
        if col_name in seen:
            seen[col_name] += 1
            new_header.append(f"{col_name}.{seen[col_name]}")
        else:
            seen[col_name] = 0
            new_header.append(col_name)
    return new_header

def load_worksheet_as_df(worksheet):
    """ワークシートを読み込み、ヘッダーを処理してDataFrameとして返す"""
    all_values = worksheet.get_all_values()
    if not all_values:
        return pd.DataFrame()
    header = get_unique_header(all_values[0])
    data = all_values[1:]
    return pd.DataFrame(data, columns=header)

# ==============================================================================
# 各処理ステップの関数
# ==============================================================================

def step1_update_master_with_wages(gc):
    """[ステップ1] 最新のコーチ時給をマスターシートに転記する"""
    print("\n--- [ステップ1] 最新のコーチ時給をマスターシートに転記 ---")
    try:
        sh_source = gc.open_by_url(SOURCE_WAGE_URL)
        latest_date = None
        latest_sheet = None
        for sheet in sh_source.worksheets():
            try:
                sheet_date = datetime.datetime.strptime(f"20{sheet.title}", "%Y/%m").date()
                if latest_date is None or sheet_date > latest_date:
                    latest_date = sheet_date
                    latest_sheet = sheet
            except ValueError:
                continue

        if latest_sheet is None:
            raise ValueError("日付形式（YY/MM）のシートが見つかりませんでした。")
        print(f"✅ 最新の時給シート '{latest_sheet.title}' を特定しました。")

        df_source = load_worksheet_as_df(latest_sheet)
        if df_source.empty or len(df_source.columns) < 44:
             raise ValueError("ソースシートの列数が不足しているか、データが空です。")
        staff_col_index = 2 # C列
        wage_col_index = 43 # AR列
        df_to_write = df_source.iloc[:, [staff_col_index, wage_col_index]].copy()
        df_to_write.columns = ['スタッフ名', '時給'] # ヘッダーを明確にする
        df_to_write = df_to_write.iloc[4:].reset_index(drop=True) # 5行目からをデータとして扱う

        sh_dest = gc.open_by_url(TARGET_SPREADSHEET_URL)
        ws_master = sh_dest.worksheet(MASTER_SHEET_NAME)

        # マスターシートの既存データを読み込み
        master_all_values = ws_master.get_all_values()
        if not master_all_values:
             raise ValueError("マスターシートが空です。")

        # 既存のマスターデータから、時給データを更新する部分を特定
        # 例として、スタッフ名の列（D列）と時給の列（E列）を更新対象とする
        master_staff_col_index = 3 # D列 (0始まり)
        master_wage_col_index = 4 # E列 (0始まり)

        # 更新用のデータリストを作成
        update_data = []
        # 既存のマスターデータを行ごとに処理
        for row_index, row in enumerate(master_all_values):
            if row_index == 0: # ヘッダー行はスキップ、または必要に応じて処理
                # ヘッダー行が必要な場合はここで処理
                continue
            if len(row) > master_staff_col_index:
                staff_name = row[master_staff_col_index].strip()
                if staff_name:
                    # 最新の時給データからスタッフ名で検索
                    wage_row = df_to_write[df_to_write['スタッフ名'] == staff_name]
                    if not wage_row.empty:
                        # 該当するスタッフが見つかった場合、最新の時給を取得
                        latest_wage = wage_row.iloc[0]['時給']
                        # 更新対象の行データを作成（E列のみ更新）
                        # 行の長さを確認し、必要に応じて拡張
                        current_row_length = len(row)
                        if current_row_length <= master_wage_col_index:
                            row.extend([''] * (master_wage_col_index - current_row_length + 1))
                        row[master_wage_col_index] = latest_wage
                    # 更新データリストに追加
                    update_data.append(row)
                else:
                     # スタッフ名が空の場合は既存の行をそのまま追加（またはスキップ）
                     update_data.append(row)
            else:
                 # D列が存在しない行は既存の行をそのまま追加
                 update_data.append(row)


        # 更新データをマスターシートに書き込み（シート全体を更新）
        # 既存のシートをクリアして新しいデータで書き込むか、指定範囲を更新するか選択
        # シンプルにシート全体をクリアして新しいデータで書き込む
        ws_master.clear()
        ws_master.update([get_unique_header(master_all_values[0])] + update_data) # 元のヘッダーを保持

        print(f"✅ マスターシートのD列とE列に最新の時給データを書き込みました。")
        return True
    except Exception as e:
        print(f"❌ ステップ1でエラーが発生しました: {e}")
        return False


def step2_reset_looker_sheet(sh):
    """[ステップ2] lookerシートをクリアし、importrange数式を再設定する"""
    print(f"\n--- [ステップ2] '{LOOKER_SHEET_NAME}' シートをIMPORTRANGEで更新 ---")
    try:
        worksheet = sh.worksheet(LOOKER_SHEET_NAME)
        worksheet.clear()
        # worksheet.update('A1', [[IMPORTRANGE_FORMULA]], raw=False) # DeprecationWarning 対応
        worksheet.update(range_name='A1', values=[[IMPORTRANGE_FORMULA]], raw=False)
        print(f"✅ '{LOOKER_SHEET_NAME}' シートをクリアし、A1セルに数式を再設定しました。")
        return True
    except Exception as e:
        print(f"❌ ステップ2でエラーが発生しました: {e}")
        return False

def step3_prepare_c_sheets_columns(sh):
    """[ステップ3] cシート群に必要な列が存在するか確認し、なければ追加する"""
    print("\n--- [ステップ3] 'c'で始まるシートの列整備 ---")
    try:
        print("IMPORTRANGEの反映を待機しています...")
        time.sleep(10) # IMPORTRANGEの反映待機時間を延長
        looker_ws = sh.worksheet(LOOKER_SHEET_NAME)
        looker_header = get_unique_header(looker_ws.row_values(1))

        required_cols_set = set(looker_header + [COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY])

        c_sheets = [ws for ws in sh.worksheets() if ws.title.startswith('c')]
        for c_sheet in c_sheets:
            c_header = c_sheet.row_values(1)
            c_header_set = set(get_unique_header(c_header))
            missing_cols = list(required_cols_set - c_header_set)

            if missing_cols:
                print(f"シート '{c_sheet.title}' に不足列 {missing_cols} を追加します。")
                # 現在の列数を取得
                current_col_count = c_sheet.col_count
                # 不足列を追加
                c_sheet.add_cols(len(missing_cols))
                # ヘッダーを更新
                for i, col_name in enumerate(missing_cols):
                    # c_sheet.update_cell(1, num_existing_cols + i + 1, col_name) # DeprecationWarning 対応
                    c_sheet.update_cell(row=1, col=current_col_count + i + 1, value=col_name)


        print("✅ 全ての'c'シートの列整備が完了しました。")
        return True
    except Exception as e:
        print(f"❌ ステップ3でエラーが発生しました: {e}")
        return False

def step4_calculate_and_update_c_sheets(sh):
    """[ステップ4] マスターデータを基に、cシート群の計算を行い更新する"""
    print("\n--- [ステップ4] 'c'シートの計算と更新 ---")
    try:
        ws_master = sh.worksheet(MASTER_SHEET_NAME)
        df_master = load_worksheet_as_df(ws_master)

        # 標準時間ルックアップを作成 (プロセス分類 -> 標準時間)
        # マスターシートのA列(プロセス分類)とB列(標準時間)を使用
        std_time_lookup = pd.Series(
            pd.to_numeric(df_master.iloc[:, 1], errors='coerce').values,
            index=df_master.iloc[:, 0]
        ).to_dict()
        print(f"✅ 標準時間ルックアップを作成しました ({len(std_time_lookup)}件)。")

        # 時給ルックアップを作成 (スタッフ名 -> 時給)
        # マスターシートのD列(スタッフ名)とE列(時給)を使用
        # E列の時給は文字列の場合があるので、数値に変換し、無効な値はNaNにする
        df_master_wage = df_master.iloc[:, [3, 4]].dropna().copy()
        df_master_wage.columns = ['スタッフ名', '時給'] # ヘッダーを明確にする
        df_master_wage['時給'] = df_master_wage['時給'].astype(str).str.replace('¥', '').str.replace(',', '').str.strip()
        wage_lookup = pd.Series(
            pd.to_numeric(df_master_wage['時給'], errors='coerce').values,
            index=df_master_wage['スタッフ名']
        ).to_dict()
        print(f"✅ 時給ルックアップを作成しました ({len(wage_lookup)}件)。")


        c_sheets = [ws for ws in sh.worksheets() if ws.title.startswith('c')]
        for c_sheet in c_sheets:
            print(f"  > シート '{c_sheet.title}' の処理中...")
            df_c = load_worksheet_as_df(c_sheet)
            if df_c.empty:
                print(f"    シート '{c_sheet.title}' は空です。スキップします。")
                continue

            # 必要な列が存在するか確認し、列インデックスを取得
            if COL_REPORTER not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_REPORTER}' 列がありません。スキップします。")
                 continue

            # 必要な計算列が存在するか確認し、なければエラーメッセージ
            if COL_STD_TIME not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_STD_TIME}' 列がありません。標準時間計算をスキップします。")
            if COL_WORK_TIME not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_WORK_TIME}' 列がありません。給与計算をスキップします。")
            if COL_HOURLY_WAGE not in df_c.columns:
                 print(f"    シート '{c_sheet.title}' に '{COL_HOURLY_WAGE}' 列がありません。時給・給与計算をスキップします。")
                 continue # 時給列がないと給与も計算できないためスキップ

            # 数値変換 (エラーがあればNaN)
            df_c[COL_WORK_TIME] = pd.to_numeric(df_c[COL_WORK_TIME], errors='coerce')
            if COL_STD_TIME in df_c.columns:
                df_c[COL_STD_TIME] = pd.to_numeric(df_c[COL_STD_TIME], errors='coerce')

            # 時給をルックアップで取得し、DataFrameに代入
            df_c[COL_HOURLY_WAGE] = df_c[COL_REPORTER].map(wage_lookup)
            df_c[COL_HOURLY_WAGE] = pd.to_numeric(df_c[COL_HOURLY_WAGE], errors='coerce')
            print(f"    シート '{c_sheet.title}' の時給を取得しました。")

            # 給与を計算 (作業時間 * 時給 / 60, 小数点以下切り捨て)
            # NaNを含む行は計算しない
            valid_rows = df_c[COL_WORK_TIME].notna() & df_c[COL_HOURLY_WAGE].notna()
            df_c.loc[valid_rows, COL_SALARY] = np.floor(
                df_c.loc[valid_rows, COL_WORK_TIME] * df_c.loc[valid_rows, COL_HOURLY_WAGE] / 60
            )
            # 計算できなかった行 (NaN) は空白に設定
            df_c[COL_SALARY] = df_c[COL_SALARY].fillna('')
            print(f"    シート '{c_sheet.title}' の給与を計算しました。")


            # 差（分）の計算
            if all(c in df_c.columns for c in [COL_WORK_TIME, COL_STD_TIME]) and COL_DIFF_MINUTES in df_c.columns:
                df_c[COL_DIFF_MINUTES] = df_c[COL_WORK_TIME] - df_c[COL_STD_TIME]
                print(f"    シート '{c_sheet.title}' の差（分）を計算しました。")


            # 更新する列のみを選択して書き込み
            cols_to_update = [COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY]
            # 実際に存在する列のみを対象とする
            existing_cols_to_update = [col for col in cols_to_update if col in df_c.columns]

            if existing_cols_to_update:
                # 更新対象のDataFrameを作成（既存の列はそのまま、計算した列は更新）
                df_for_write = df_c.copy()
                # NaNを空白に変換して書き込みに備える
                df_for_write[existing_cols_to_update] = df_for_write[existing_cols_to_update].fillna('')

                # ヘッダー行を含めて、DataFrame全体を書き込み
                c_sheet.clear()
                set_with_dataframe(c_sheet, df_for_write, include_index=False)
                print(f"    シート '{c_sheet.title}' を更新しました。")
            else:
                 print(f"    シート '{c_sheet.title}' に更新対象の列がありませんでした。")


        print("✅ 全ての'c'シートの計算と更新が完了しました。")
        return True
    except Exception as e:
        print(f"❌ ステップ4でエラーが発生しました: {e}")
        return False

def step5_create_summary_sheet(sh):
    """[ステップ5] lookerとcシート群からデータを集約し、`集計`シートを作成する"""
    print(f"\n--- [ステップ5] '{SUMMARY_SHEET_NAME}' シートの作成 ---")
    try:
        ws_looker = sh.worksheet(LOOKER_SHEET_NAME)
        df_looker = load_worksheet_as_df(ws_looker)
        if not df_looker.empty and COL_BILLING_MONTH in df_looker.columns:
            # '記帳該当月' 列が数値に変換できる行のみを抽出
            df_looker_filtered = df_looker[pd.to_numeric(df_looker[COL_BILLING_MONTH], errors='coerce').notna()].copy()
        else:
            df_looker_filtered = pd.DataFrame(columns=df_looker.columns.tolist() + [COL_STD_TIME, COL_DIFF_MINUTES]) # 存在しない列を追加して空のDataFrameを作成
        print(f"✅ '{LOOKER_SHEET_NAME}' シートから {len(df_looker_filtered)} 件のデータを抽出しました。")

        all_c_dfs = []
        c_sheets = [ws for ws in sh.worksheets() if ws.title.startswith('c')]
        key_cols = [COL_REPORTER, COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_CUSTOMER_NAME, COL_PROCESS_CATEGORY, COL_TASK_DETAIL]
        sum_cols = [COL_WORK_TIME, COL_SALARY, COL_BILLING, COL_STD_TIME, COL_DIFF_MINUTES]

        for c_sheet in c_sheets:
            df_c = load_worksheet_as_df(c_sheet)
            if df_c.empty: continue
            if COL_BILLING_MONTH not in df_c.columns: continue
            # '記帳該当月' 列が数値に変換できる行のみを抽出
            df_c_filtered = df_c[pd.to_numeric(df_c[COL_BILLING_MONTH], errors='coerce').notna()].copy()
            if df_c_filtered.empty: continue

            # 集計対象列を数値に変換 (エラーはNaN)
            for col in sum_cols:
                if col in df_c_filtered.columns:
                    df_c_filtered[col] = pd.to_numeric(df_c_filtered[col], errors='coerce')

            # 存在するキー列のみを使用
            existing_key_cols = [col for col in key_cols if col in df_c_filtered.columns]
            if not existing_key_cols:
                print(f"⚠️ シート '{c_sheet.title}' に集計に必要なキー列がありません。スキップします。")
                continue

            # 集計辞書を作成（存在する集計対象列のみ）
            agg_dict = {col: 'sum' for col in sum_cols if col in df_c_filtered.columns}
            # 時給は平均値または最初の値など、適切な集計方法を選択（ここでは最初の値とする）
            if COL_HOURLY_WAGE in df_c_filtered.columns:
                 agg_dict[COL_HOURLY_WAGE] = 'first'


            # 存在するキー列と集計対象列のみを含むDataFrameを作成して集計
            cols_for_agg = existing_key_cols + list(agg_dict.keys())
            df_c_agg = df_c_filtered[cols_for_agg].groupby(existing_key_cols, as_index=False).agg(agg_dict)
            all_c_dfs.append(df_c_agg)

        # lookerデータとcシート群のデータを結合
        # 結合前に両方のDataFrameに必要な全列が存在するように調整
        all_cols = list(set(df_looker_filtered.columns.tolist() + [col for df in all_c_dfs for col in df.columns]))
        df_looker_aligned = df_looker_filtered.reindex(columns=all_cols)
        c_combined_aligned = pd.concat(all_c_dfs, ignore_index=True).reindex(columns=all_cols) if all_c_dfs else pd.DataFrame(columns=all_cols)


        df_summary = pd.concat([df_looker_aligned, c_combined_aligned], ignore_index=True)

        # マスターシートから標準時間ルックアップを取得
        ws_master = sh.worksheet(MASTER_SHEET_NAME)
        df_master = load_worksheet_as_df(ws_master)
        std_time_lookup = pd.Series(
            pd.to_numeric(df_master.iloc[:, 1], errors='coerce').values,
            index=df_master.iloc[:, 0]
        ).to_dict()

        # 集計シートの「プロセス分類」列を基に「標準時間」列を更新
        if COL_PROCESS_CATEGORY in df_summary.columns and COL_STD_TIME in df_summary.columns:
            df_summary[COL_STD_TIME] = df_summary[COL_PROCESS_CATEGORY].map(std_time_lookup)
            # 標準時間列を数値型に変換（エラーはNaN）
            df_summary[COL_STD_TIME] = pd.to_numeric(df_summary[COL_STD_TIME], errors='coerce')

        # 「標準時間」が記載がある行について、「作業時間」-"標準時間"の結果を"差（分）"に記載
        if all(c in df_summary.columns for c in [COL_WORK_TIME, COL_STD_TIME, COL_DIFF_MINUTES]):
            # 作業時間と標準時間列を数値型に変換（エラーはNaN）
            df_summary[COL_WORK_TIME] = pd.to_numeric(df_summary[COL_WORK_TIME], errors='coerce')
            df_summary[COL_STD_TIME] = pd.to_numeric(df_summary[COL_STD_TIME], errors='coerce')

            # 標準時間がある行で差分を計算
            has_std_time = df_summary[COL_STD_TIME].notna()
            df_summary.loc[has_std_time, COL_DIFF_MINUTES] = df_summary.loc[has_std_time, COL_WORK_TIME] - df_summary.loc[has_std_time, COL_STD_TIME]
            # 標準時間がない行の差分はNaNに設定
            df_summary.loc[~has_std_time, COL_DIFF_MINUTES] = np.nan
            print(f"✅ '{SUMMARY_SHEET_NAME}' シートの差（分）を計算しました。")


        try:
            ws_summary = sh.worksheet(SUMMARY_SHEET_NAME)
            ws_summary.clear()
        except gspread.exceptions.WorksheetNotFound:
            ws_summary = sh.add_worksheet(title=SUMMARY_SHEET_NAME, rows="1000", cols="30")

        # NaNを空白に変換して書き込み
        df_for_write = df_summary.fillna('')
        set_with_dataframe(ws_summary, df_for_write, include_index=False)
        print(f"✅ '{SUMMARY_SHEET_NAME}' シートに {len(df_summary)} 件のデータを書き込みました。")
        return True
    except Exception as e:
        print(f"❌ ステップ5でエラーが発生しました: {e}")
        return False

def step6_create_final_report_sheet(sh):
    """[ステップ6] `集計`シートを基に`集計結果（月別顧客別）`シートを作成し、書式設定する"""
    print(f"\n--- [ステップ6] '{FINAL_REPORT_SHEET_NAME}' シートの作成と書式設定 ---")
    try:
        ws_summary = sh.worksheet(SUMMARY_SHEET_NAME)
        # 集計シートのフィルタを削除
        requests = []
        requests.append({
            "clearBasicFilter": {
                "sheetId": ws_summary.id
            }
        })
        sh.batch_update({"requests": requests})
        print(f"✅ '{SUMMARY_SHEET_NAME}' シートのフィルタを削除しました。")

        df_summary = load_worksheet_as_df(ws_summary)
        if df_summary.empty:
            print(f"⚠️ '{SUMMARY_SHEET_NAME}' シートが空のため、処理をスキップします。")
            return True

        # メインの集計グループ（月別、顧客番号別、プロセス分類別）
        # 報告者もグループ化のキーに含めることで、報告者ごとの集計を維持
        main_group_cols = [COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_PROCESS_CATEGORY, COL_REPORTER]
        # 小計のグループ（記帳該当月、顧客番号） - ユーザーの要望に合わせて変更
        subtotal_group_cols = [COL_BILLING_MONTH, COL_CUSTOMER_NO]

        # 集計対象とする数値列のリスト（損益計算前に定義）
        sum_cols_base = [COL_WORK_TIME, COL_SALARY, COL_BILLING, COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE] # 時給も合計対象に含める


        # 損益列を計算 (集計前に行う)
        # 必要な列が存在するか確認し、数値に変換
        if all(c in df_summary.columns for c in [COL_BILLING, COL_SALARY]):
             df_summary[COL_BILLING] = pd.to_numeric(df_summary[COL_BILLING], errors='coerce').fillna(0)
             df_summary[COL_SALARY] = pd.to_numeric(df_summary[COL_SALARY], errors='coerce').fillna(0)
             # 損益列を計算
             df_summary[COL_PROFIT_LOSS] = df_summary[COL_BILLING] - df_summary[COL_SALARY]
             print(f"✅ '{SUMMARY_SHEET_NAME}' シートに '{COL_PROFIT_LOSS}' 列を計算しました。")
             # 損益列を集計対象列に追加
             sum_cols = sum_cols_base + [COL_PROFIT_LOSS] # 損益列を追加したリスト
        else:
             print(f"⚠️ '{COL_BILLING}' または '{COL_SALARY}' 列が集計シートに存在しないため、'{COL_PROFIT_LOSS}' 列は計算されません。")
             sum_cols = sum_cols_base # 損益列がない場合は基本のリストを使用


        # 実際に集計シートに存在するキー列と集計対象列を特定
        existing_main_group_cols = [c for c in main_group_cols if c in df_summary.columns]
        existing_subtotal_group_cols = [c for c in subtotal_group_cols if c in df_summary.columns]
        existing_sum_cols = [c for c in sum_cols if c in df_summary.columns]

        # 集計辞書を作成
        agg_dict = {}
        # 集計対象列は 'sum'
        for col in existing_sum_cols:
             agg_dict[col] = 'sum'

        # 集計対象列およびメイングループ化列以外の列は 'first' （Unnamed_列は除く）
        for col in df_summary.columns:
             if col not in existing_main_group_cols and col not in existing_sum_cols and not col.startswith('Unnamed_'):
                 agg_dict[col] = 'first'


        if not existing_main_group_cols:
             raise ValueError("集計に必要なメインキー列が集計シートにありません。")

        # 集計対象列を数値に変換 (エラーは0に変換) - 集計前に実施
        for col in existing_sum_cols:
             if col in df_summary.columns: # 念のため再確認
                df_summary[col] = pd.to_numeric(df_summary[col], errors='coerce').fillna(0)


        # agg_dictが空の場合のチェック
        if not agg_dict:
             print("⚠️ 集計対象となる列がありません。集計処理をスキップします。")
             df_aggregated = df_summary.copy() # 集計せずにそのまま使用
        else:
            # 存在するメインキー列でグループ化し、集計を実行
            df_aggregated = df_summary.groupby(existing_main_group_cols, as_index=False).agg(agg_dict)


        # 記帳該当月、顧客番号、報告者でソート
        sort_cols = [c for c in [COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_REPORTER] if c in df_aggregated.columns]
        if sort_cols:
            df_aggregated = df_aggregated.sort_values(by=sort_cols)

        # 小計行を作成して結合
        final_dfs = []
        # 小計グループ（記帳該当月、顧客番号）でループして小計行を作成
        if existing_subtotal_group_cols:
            # 小計グループの組み合わせを取得
            subtotal_groups = df_aggregated.groupby(existing_subtotal_group_cols)

            # 集計されたDataFrameの実際の列名を取得
            aggregated_columns = df_aggregated.columns.tolist()


            for name, group in subtotal_groups:
                # 各グループのデータを追加
                final_dfs.append(group)

                # 小計行のデータを作成
                # ユーザーが指定した小計対象列のみを合計
                # 小計対象列を確実に数値に変換してから合計
                subtotal_cols_to_sum = [col for col in [COL_WORK_TIME, COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY, COL_BILLING, COL_PROFIT_LOSS] if col in group.columns]
                group_numeric = group[subtotal_cols_to_sum].apply(pd.to_numeric, errors='coerce').fillna(0)
                subtotal = group_numeric.sum()


                # 小計行のデータフレームを作成（集計済みDataFrameと同じ列構成）
                subtotal_row_data = subtotal.to_dict()
                subtotal_row_df = pd.DataFrame([subtotal_row_data]).reindex(columns=aggregated_columns).fillna('')


                # 小計グループの列名に '小計' または対応するグループ値を設定
                for i, col_name in enumerate(existing_subtotal_group_cols):
                     if col_name in subtotal_row_df.columns:
                         # nameがタプルの場合、要素数が合うか確認
                         if isinstance(name, tuple) and i < len(name):
                              subtotal_row_df[col_name] = name[i] # グループの値を設定
                         else:
                              # nameが単一値の場合
                              subtotal_row_df[col_name] = name # グループの値を設定


                # 顧客名列に '小計' を設定 (もし存在するなら)
                if COL_CUSTOMER_NAME in subtotal_row_df.columns:
                     subtotal_row_df[COL_CUSTOMER_NAME] = '小計'

                # その他のメイングループ化列で、小計グループに含まれない列は空白に設定
                other_main_group_cols = [col for col in existing_main_group_cols if col not in existing_subtotal_group_cols]
                for col in other_main_group_cols:
                     if col in subtotal_row_df.columns:
                         subtotal_row_df[col] = ''

                # タスク詳細、報告者列も小計行では空白にする
                if COL_TASK_DETAIL in subtotal_row_df.columns:
                     subtotal_row_df[COL_TASK_DETAIL] = ''
                # 報告者列は小計行では空白にする
                if COL_REPORTER in subtotal_row_df.columns:
                     subtotal_row_df[COL_REPORTER] = ''


                final_dfs.append(subtotal_row_df)

             # 最後に全体の合計行を追加することも検討（ここではスキップ）

        else:
             # 小計グループ列がない場合は、集計済みデータそのまま
             final_dfs.append(df_aggregated)


        df_final = pd.concat(final_dfs, ignore_index=True) if final_dfs else pd.DataFrame(columns=df_aggregated.columns)

        # --- 記帳該当月ごとの合計行を追加 ---
        monthly_total_dfs = []
        # 記帳該当月でグループ化
        if COL_BILLING_MONTH in df_final.columns and COL_CUSTOMER_NAME in df_final.columns:
             # 小計行のみを抽出して、記帳該当月でグループ化し合計を計算
             df_subtotals = df_final[df_final[COL_CUSTOMER_NAME].astype(str) == '小計'].copy()

             if not df_subtotals.empty:
                 # 合計対象列を数値に変換 (エラーは0)
                 # 月合計で合計する列を明示的に指定
                 monthly_sum_cols = [col for col in [COL_WORK_TIME, COL_STD_TIME, COL_DIFF_MINUTES, COL_SALARY, COL_BILLING, COL_PROFIT_LOSS] if col in df_subtotals.columns] # 時給は月合計では合計しない

                 # 月合計対象列を数値に変換 (エラーは0)
                 for col in monthly_sum_cols:
                     if col in df_subtotals.columns:
                         df_subtotals[col] = pd.to_numeric(df_subtotals[col], errors='coerce').fillna(0)

                 monthly_totals = df_subtotals.groupby(COL_BILLING_MONTH)[monthly_sum_cols].sum().reset_index()

                 # 月ごとの合計行を作成し、元のデータに挿入
                 inserted_dfs = []
                 # 元のデータを記帳該当月でグループ化
                 for month, group in df_final.groupby(COL_BILLING_MONTH, sort=False):
                      inserted_dfs.append(group) # 月のデータ追加
                      # その月の合計行を検索
                      month_total_row = monthly_totals[monthly_totals[COL_BILLING_MONTH] == month]
                      if not month_total_row.empty:
                           # 合計行のDataFrameを作成
                           total_row_data = month_total_row.iloc[0].to_dict()
                           # 他の列は空白または適切な値で埋める
                           total_row_data[COL_CUSTOMER_NAME] = '月合計' # 顧客名列に「月合計」
                           total_row_data[COL_CUSTOMER_NO] = '' # 顧客番号列は空白
                           total_row_data[COL_PROCESS_CATEGORY] = '' # プロセス分類列は空白
                           total_row_data[COL_TASK_DETAIL] = '' # タスク詳細列は空白
                           total_row_data[COL_REPORTER] = '' # 報告者列は空白
                           total_row_data[COL_HOURLY_WAGE] = '' # 時給列は空白にする

                           # 元のdf_finalと同じ列順でDataFrameを作成
                           total_row_df = pd.DataFrame([total_row_data]).reindex(columns=df_final.columns).fillna('')
                           inserted_dfs.append(total_row_df) # 合計行追加

                 df_final_with_totals = pd.concat(inserted_dfs, ignore_index=True) if inserted_dfs else pd.DataFrame(columns=df_final.columns)
                 df_final = df_final_with_totals # データフレームを更新
                 print(f"✅ '{FINAL_REPORT_SHEET_NAME}' シートに記帳該当月ごとの合計行を追加しました。")
             else:
                  print(f"⚠️ '{FINAL_REPORT_SHEET_NAME}' シートに'小計'行が見つからなかったため、月合計は計算されません。")


        # 表示列の順序を定義（損益列も含む）
        display_cols_order = [COL_BILLING_MONTH, COL_CUSTOMER_NO, COL_CUSTOMER_NAME, COL_PROCESS_CATEGORY, COL_TASK_DETAIL, COL_REPORTER, COL_WORK_TIME, COL_STD_TIME, COL_DIFF_MINUTES, COL_HOURLY_WAGE, COL_SALARY, COL_BILLING]
        if COL_PROFIT_LOSS in df_final.columns and COL_PROFIT_LOSS not in display_cols_order:
             # 損益列を「請求」列の直後に追加
             try:
                 billing_index = display_cols_order.index(COL_BILLING)
                 display_cols_order.insert(billing_index + 1, COL_PROFIT_LOSS)
             except ValueError:
                 # 「請求」列がない場合は最後に近い位置に追加
                 display_cols_order.append(COL_PROFIT_LOSS)


        # 表示列の順序を調整 (存在する列のみ)
        existing_display_cols = [c for c in display_cols_order if c in df_final.columns]
        # もし既存の表示列リストが空の場合、DataFrameの全ての列を使用
        if not existing_display_cols:
             existing_display_cols = df_final.columns.tolist()

        df_final = df_final[existing_display_cols]


        try:
            ws_final = sh.worksheet(FINAL_REPORT_SHEET_NAME)
            # 集計結果シートのフィルタを削除
            requests = []
            requests.append({
                "clearBasicFilter": {
                    "sheetId": ws_final.id
                }
            })
            sh.batch_update({"requests": requests})
            print(f"✅ '{FINAL_REPORT_SHEET_NAME}' シートのフィルタを削除しました。")

            # 既存のシートをクリアする前に、すべてのセル色をクリアする
            ws_final.format('A1:Z', {'backgroundColor': {'red': 1.0, 'green': 1.0, 'blue': 1.0}}) # 白で塗りつぶし
            ws_final.clear()
        except gspread.exceptions.WorksheetNotFound:
            ws_final = sh.add_worksheet(title=FINAL_REPORT_SHEET_NAME, rows="1000", cols="30")

        # NaNを空白に変換して書き込み
        df_for_write = df_final.fillna('')
        set_with_dataframe(ws_final, df_for_write, include_index=False)
        print(f"✅ '{FINAL_REPORT_SHEET_NAME}' に {len(df_final)} 件のデータを書き込みました。")

        # --- 書式設定（APIエラー修正版） ---
        requests = []
        last_row = len(df_final) + 1 # ヘッダー行を含むため+1
        last_col = len(df_final.columns)

        # 1. 罫線設定のリクエスト
        border_style = {"style": "SOLID", "width": 1, "color": {"red": 0.0, "green": 0.0, "blue": 0.0}}
        # シート全体の罫線
        requests.append({
            "updateBorders": {
                "range": {"sheetId": ws_final.id, "startRowIndex": 0, "endRowIndex": last_row, "startColumnIndex": 0, "endColumnIndex": last_col},
                "top": border_style, "bottom": border_style, "left": border_style, "right": border_style,
                "innerHorizontal": border_style, "innerVertical": border_style,
            }
        })


        # 2. 小計行と月合計行の背景色設定のリクエスト (最終行から2行目に向かって処理)
        # 小計行と月合計行のインデックスを特定 (ヘッダー行を考慮して+1)
        # COL_CUSTOMER_NAME (顧客名) 列に '小計' または '月合計' が含まれる行を特定
        # str() で文字列に変換してから比較することで、NaNなどのエラーを防ぐ
        # 最終行から逆順に処理するために reversed() を使用
        colored_rows_indices = [
            i + 1 for i, row in df_final.iterrows()
            if COL_CUSTOMER_NAME in row and isinstance(row[COL_CUSTOMER_NAME], str) and (row[COL_CUSTOMER_NAME] == '小計' or row[COL_CUSTOMER_NAME] == '月合計')
        ]

        # 最終行から逆順に処理するため、インデックスを逆順にする
        colored_rows_indices.sort(reverse=True)


        if colored_rows_indices:
            for row_index in colored_rows_indices:
                # APIリクエストを作成
                requests.append({
                    "repeatCell": {
                        "range": {"sheetId": ws_final.id, "startRowIndex": row_index, "endRowIndex": row_index + 1, "startColumnIndex": 0, "endColumnIndex": last_col}, # endRowIndexは排他的なので+1
                        "cell": {"userEnteredFormat": {"backgroundColor": {"red": 1.0, "green": 1.0, "blue": 0.8}}}, # 薄い黄色
                        "fields": "userEnteredFormat.backgroundColor"
                    }
                })


        # 3. リクエストをまとめて実行
        if requests:
            sh.batch_update({"requests": requests})
            print(f"✅ 書式設定（罫線、小計行、月合計行の背景色）を適用しました。")

        return True
    except Exception as e:
        print(f"❌ ステップ6でエラーが発生しました: {e}")
        return False


# ==============================================================================
# メイン処理
# ==============================================================================
def main():
    """全ての処理を順番に実行するメイン関数"""
    print("====== データ処理を開始します ======")

    # URL設定の確認
    if SOURCE_WAGE_URL == "YOUR_SOURCE_SPREADSHEET_URL_HERE":
        print("❌ 設定エラー: SOURCE_WAGE_URLを実際のスプレッドシートURLに変更してください。")
        return
    if TARGET_SPREADSHEET_URL == "YOUR_TARGET_SPREADSHEET_URL_HERE":
        print("❌ 設定エラー: TARGET_SPREADSHEET_URLを実際のスプレッドシートURLに変更してください。")
        return
    if IMPORTRANGE_FORMULA == '=importrange("YOUR_LOOKER_SPREADSHEET_URL_HERE","Looker!A1:h")':
        print("❌ 設定エラー: IMPORTRANGE_FORMULAのURLを実際のスプレADルシートURLに変更してください。")
        return

    gc = get_gspread_client()
    if not gc: return

    if not step1_update_master_with_wages(gc): return

    try:
        sh = gc.open_by_url(TARGET_SPREADSHEET_URL)
    except Exception as e:
        print(f"❌ メインの処理対象スプレッドシートが開けません: {e}")
        return

    if not step2_reset_looker_sheet(sh): return
    if not step3_prepare_c_sheets_columns(sh): return
    if not step4_calculate_and_update_c_sheets(sh): return
    if not step5_create_summary_sheet(sh): return
    if not step6_create_final_report_sheet(sh): return

    print("\n🎉🎉🎉 全ての処理が正常に完了しました！ 🎉🎉🎉")
    print(f"最終確認はこちら: {TARGET_SPREADSHEET_URL}")

# --- スクリプトの実行 ---
main()

====== データ処理を開始します ======
✅ Google スプレッドシートへの認証が完了しました。

--- [ステップ1] 最新のコーチ時給をマスターシートに転記 ---
✅ 最新の時給シート '25/07' を特定しました。
✅ マスターシートのD列とE列に最新の時給データを書き込みました。

--- [ステップ2] 'looker' シートをIMPORTRANGEで更新 ---
✅ 'looker' シートをクリアし、A1セルに数式を再設定しました。

--- [ステップ3] 'c'で始まるシートの列整備 ---
IMPORTRANGEの反映を待機しています...
✅ 全ての'c'シートの列整備が完了しました。

--- [ステップ4] 'c'シートの計算と更新 ---
✅ 標準時間ルックアップを作成しました (29件)。
✅ 時給ルックアップを作成しました (32件)。
  > シート 'c大島' の処理中...
    シート 'c大島' の時給を取得しました。
    シート 'c大島' の給与を計算しました。
    シート 'c大島' の差（分）を計算しました。
    シート 'c大島' を更新しました。
  > シート 'c矢頭' の処理中...
    シート 'c矢頭' の時給を取得しました。
    シート 'c矢頭' の給与を計算しました。
    シート 'c矢頭' の差（分）を計算しました。
    シート 'c矢頭' を更新しました。
  > シート 'c尾崎' の処理中...
    シート 'c尾崎' の時給を取得しました。
    シート 'c尾崎' の給与を計算しました。
    シート 'c尾崎' の差（分）を計算しました。
    シート 'c尾崎' を更新しました。
  > シート 'c加藤' の処理中...
    シート 'c加藤' の時給を取得しました。
    シート 'c加藤' の給与を計算しました。
    シート 'c加藤' の差（分）を計算しました。
    シート 'c加藤' を更新しました。
✅ 全ての'c'シートの計算と更新が完了しました。

--- [ステップ5] '集計' シートの作成 ---
✅ 'looker' シートから 38354 件のデータを抽出しました。
✅ '集計' 